## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

- **Model**: `speech_to_text_medical`
- **Model Description**: This pipeline converts audio to text, providing timestamps for each transcribed segment, and assigns speaker IDs. Then, it extracts clinical entities from the transcribed text, assigns assertion statuses, and establishes relationships between the extracted entities.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "speech-to-text-medical"

content_type = "application/octet-stream"

real_time_inference_instance_type = "ml.m4.2xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Perform real-time inference

In [6]:
import json
import pandas as pd
import os
import boto3
import shutil

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(audio_file):
    with open(audio_file, "rb") as file:
        audio_data = file.read()

    i = 1
    input_extension = os.path.splitext(audio_file)[-1]
    input_file_name = f'inputs/real-time/input{i}{input_extension}'

    while os.path.exists(input_file_name):
        i += 1
        input_file_name = f'inputs/real-time/input{i}{input_extension}'

    output_file_name = f'outputs/real-time/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    shutil.copy2(audio_file, input_file_name)

    # Assuming s3_client is defined and used correctly
    validation_input_file_path = f"{model_name}/validation-input/real-time/{os.path.basename(input_file_name)}"
    s3_client.upload_file(audio_file, s3_bucket, validation_input_file_path)

    # Assuming sm_runtime is defined and used correctly
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept="application/json",
        Body=audio_data,
    )

    # Process response
    response_data = json.loads(response["Body"].read().decode("utf-8"))

    # Save response data to file
    s3_client.put_object(Bucket=s3_bucket, Key=f"{model_name}/validation-output/real-time/{os.path.basename(output_file_name)}", Body=json.dumps(response_data).encode('UTF-8'))
    with open(output_file_name, 'w') as f_out:
        json.dump(response_data, f_out, indent=4)

    return response_data


### Initial setup

In [ ]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/products/sagemaker/models/speech_to_text_medical/inputs/batch/input1.wav \
     https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/products/sagemaker/models/speech_to_text_medical/inputs/batch/input2.wav \
     https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/products/sagemaker/models/speech_to_text_medical/inputs/batch/input3.wav

#### Example 1

In [8]:
data  = process_data_and_invoke_realtime_endpoint(audio_file = "input1.wav")

In [9]:
import pandas as pd

# Creating the DataFrame
document = pd.DataFrame([data["document"]], columns=["document"])
segments = pd.DataFrame(data["segments"])
ner_df = pd.DataFrame(data["ner_predictions"])
assertion_df = pd.DataFrame(data["assertion_predictions"])
relation_df = pd.DataFrame(data["relation_predictions"])

In [10]:
document

,document
0,"My husband has a spot on his lip that he thought was cold sore, but now it has broken open and leaks fluid all day"


In [11]:
segments

,speaker,begin,end,text
0,SPEAKER_00,0.401,7.1,"My husband has a spot on his lip that he thought was cold sore, but now it has broken open and leaks fluid all day"


In [12]:
ner_df

,ner_chunk,begin,end,ner_label,ner_confidence
0,husband,3,9,Gender,0.9994
1,spot on his lip,17,31,Symptom,0.55395
2,he,38,39,Gender,0.9979
3,cold,53,56,Modifier,0.7477
4,sore,58,61,Symptom,0.6247
5,broken open,79,89,Symptom,0.43985
6,leaks fluid,95,105,Symptom,0.7144
7,all day,107,113,RelativeDate,0.46680003


In [13]:
assertion_df

,ner_chunk,begin,end,ner_label,assertion
0,spot on his lip,17,31,Symptom,Present
1,sore,58,61,Symptom,Possible
2,broken open,79,89,Symptom,Present
3,leaks fluid,95,105,Symptom,Present


In [14]:
relation_df

,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relation,relation_confidence
0,spot on his lip,17,31,Symptom,all day,107,113,RelativeDate,O,0.9802182
1,sore,58,61,Symptom,all day,107,113,RelativeDate,O,1.0
2,broken open,79,89,Symptom,all day,107,113,RelativeDate,O,1.0
3,leaks fluid,95,105,Symptom,all day,107,113,RelativeDate,is_date_of,1.0


#### Example 2

In [15]:
data  = process_data_and_invoke_realtime_endpoint(audio_file = "input2.wav")

In [16]:
import pandas as pd

# Creating the DataFrame
document = pd.DataFrame([data["document"]], columns=["document"])
segments = pd.DataFrame(data["segments"])
ner_df = pd.DataFrame(data["ner_predictions"])
relation_df = pd.DataFrame(data["relation_predictions"])

In [17]:
document

,document
0,"Lately I've been dealing with congestion and difficulty breathing, especially while eating. After getting the influenza vaccine, I developed a fever and decreased appetite. Despite trying albuterol treatments, my symptoms haven't improved and my urine output has decreased. Hoping for a solution soon."


In [18]:
segments

,speaker,begin,end,text
0,SPEAKER_00,0.100,4.940,"Lately I've been dealing with congestion and difficulty breathing, especially while eating."
1,SPEAKER_00,5.901,10.880,"After getting the influenza vaccine, I developed a fever and decreased appetite."
2,SPEAKER_00,11.801,15.898,"Despite trying albuterol treatments, my symptoms haven't improved"
3,SPEAKER_00,16.481,18.810,and my urine output has decreased.
4,SPEAKER_00,19.693,20.999,Hoping for a solution soon.


In [19]:
ner_df

,ner_chunk,begin,end,ner_label,ner_confidence
0,congestion,30,39,Symptom,0.9786
1,difficulty breathing,45,64,Symptom,0.79995
2,influenza vaccine,110,126,Vaccine_Name,0.93735003
3,fever,143,147,VS_Finding,0.9875
4,decreased appetite,153,170,Symptom,0.84305
5,albuterol,188,196,Drug_Ingredient,0.9841
6,urine output has decreased,246,271,Symptom,0.41342497
7,solution,287,294,Form,0.9779


In [20]:
assertion_df

,ner_chunk,begin,end,ner_label,assertion
0,spot on his lip,17,31,Symptom,Present
1,sore,58,61,Symptom,Possible
2,broken open,79,89,Symptom,Present
3,leaks fluid,95,105,Symptom,Present


In [21]:
relation_df

""


### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [22]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Create an Asynchronous Inference Endpoint

To set up an asynchronous inference endpoint with Amazon SageMaker, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-create-endpoint.html).

In [23]:
from time import gmtime, strftime
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig


async_inference_config = AsyncInferenceConfig(
    output_path=f"s3://{s3_bucket}/{model_name}/validation-output/async-real-time",
    max_concurrent_invocations_per_instance=2,  # Adjust as needed
    failure_path=f"s3://{s3_bucket}/{model_name}/validation-output/async-real-time-failure",
)


In [24]:
# Deploy the model with AsyncInferenceConfig
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type,
    endpoint_name=model_name,
    async_inference_config=async_inference_config
)


-----------!

### A. Uploading the Request Payload

In [25]:
def upload_file(input_location):
    prefix = f"{model_name}/validation-input/invoke_endpoint_async"
    return sagemaker_session.upload_data(
        input_location,
        bucket=s3_bucket,
        key_prefix=prefix,
        extra_args={"ContentType": "application/octet-stream"},
    )
input_1_location = "input1.wav"
input_1_s3_location = upload_file(input_1_location)

### B. Invoke Endpoint

In [26]:
response = sm_runtime.invoke_endpoint_async(
    EndpointName="speech-to-text-medical",
    InputLocation=input_1_s3_location,
    ContentType=content_type,
    Accept="application/json",
)
output_location = response["OutputLocation"]

### C. Check Output Location

Check the output location to see if the inference has been processed. We make multiple requests (beginning of the `while True` statement in the `get_output` function) every two seconds until there is an output of the inference request:

In [27]:
import urllib, time
from botocore.exceptions import ClientError


def get_output(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    while True:
        try:
            return sagemaker_session.read_s3_file(bucket=output_url.netloc, key_prefix=output_url.path[1:])
        except ClientError as e:
            if e.response["Error"]["Code"] == "NoSuchKey":
                print("waiting for output...")
                time.sleep(2)
                continue
            raise

In [28]:
output = get_output(output_location)
output

waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...
waiting for output...


'{"document":"My husband has a spot on his lip that he thought was cold sore, but now it has broken open and leaks fluid all day","segments":[{"speaker":"SPEAKER_00","begin":0.401,"end":7.1,"text":"My husband has a spot on his lip that he thought was cold sore, but now it has broken open and leaks fluid all day"}],"ner_predictions":[{"ner_chunk":"husband","begin":3,"end":9,"ner_label":"Gender","ner_confidence":"0.9994"},{"ner_chunk":"spot on his lip","begin":17,"end":31,"ner_label":"Symptom","ner_confidence":"0.55395"},{"ner_chunk":"he","begin":38,"end":39,"ner_label":"Gender","ner_confidence":"0.9979"},{"ner_chunk":"cold","begin":53,"end":56,"ner_label":"Modifier","ner_confidence":"0.7477"},{"ner_chunk":"sore","begin":58,"end":61,"ner_label":"Symptom","ner_confidence":"0.6247"},{"ner_chunk":"broken open","begin":79,"end":89,"ner_label":"Symptom","ner_confidence":"0.43985"},{"ner_chunk":"leaks fluid","begin":95,"end":105,"ner_label":"Symptom","ner_confidence":"0.7144"},{"ner_chunk":"al

### D. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [29]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [31]:
import os

validation_file_name1 = "input1.wav"
validation_file_name2 = "input2.wav"
validation_file_name3 = "input3.wav"

validation_input_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch"
validation_output_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch"

input_dir = 'inputs/batch'
output_dir = 'outputs/batch'

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [32]:
import os
import boto3

def upload_audio_to_s3(local_file_path, model_name, s3_bucket):
    shutil.copy2(local_file_path, input_dir)
    base_file_name = os.path.basename(local_file_path)
    validation_input_file_path = f"{model_name}/validation-input/batch/{base_file_name}"
    s3_client.upload_file(local_file_path, s3_bucket, validation_input_file_path)

# List of file names to upload
validation_file_names = [validation_file_name1, validation_file_name2, validation_file_name3]

for file_name in validation_file_names:
    upload_audio_to_s3(file_name, model_name, s3_bucket)


In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_path,
)
transformer.transform(validation_input_path, content_type=content_type)
transformer.wait()

In [34]:
from urllib.parse import urlparse

def process_s3_output_and_save(validation_file_name, output_file_name):
    output_file_path = f"{output_dir}/{output_file_name}"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}/{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))

    # Define a dictionary with DataFrame objects and their names
    data_frames = {
        "Document DataFrame": pd.DataFrame([data["document"]], columns=["document"]),
        "Segments DataFrame": pd.DataFrame(data['segments']),
        "NER Predictions DataFrame": pd.DataFrame(data['ner_predictions']),
        "Assertion Predictions DataFrame": pd.DataFrame(data['assertion_predictions']),
        "Relation Predictions DataFrame": pd.DataFrame(data['relation_predictions'])
    }

    # Display and save each DataFrame
    for name, df in data_frames.items():
        print(name + ":")
        display(df)
    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)


#### Example 1

In [35]:
process_s3_output_and_save(validation_file_name1, "out1.out")

Document DataFrame:


,document
0,"My husband has a spot on his lip that he thought was cold sore, but now it has broken open and leaks fluid all day"


Segments DataFrame:


,speaker,begin,end,text
0,SPEAKER_00,0.401,7.1,"My husband has a spot on his lip that he thought was cold sore, but now it has broken open and leaks fluid all day"


NER Predictions DataFrame:


,ner_chunk,begin,end,ner_label,ner_confidence
0,husband,3,9,Gender,0.9994
1,spot on his lip,17,31,Symptom,0.55395
2,he,38,39,Gender,0.9979
3,cold,53,56,Modifier,0.7477
4,sore,58,61,Symptom,0.6247
5,broken open,79,89,Symptom,0.43985
6,leaks fluid,95,105,Symptom,0.7144
7,all day,107,113,RelativeDate,0.46680003


Assertion Predictions DataFrame:


,ner_chunk,begin,end,ner_label,assertion
0,spot on his lip,17,31,Symptom,Present
1,sore,58,61,Symptom,Possible
2,broken open,79,89,Symptom,Present
3,leaks fluid,95,105,Symptom,Present


Relation Predictions DataFrame:


,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relation,relation_confidence
0,spot on his lip,17,31,Symptom,all day,107,113,RelativeDate,O,0.9802182
1,sore,58,61,Symptom,all day,107,113,RelativeDate,O,1.0
2,broken open,79,89,Symptom,all day,107,113,RelativeDate,O,1.0
3,leaks fluid,95,105,Symptom,all day,107,113,RelativeDate,is_date_of,1.0


#### Example 2

In [36]:
process_s3_output_and_save(validation_file_name2, "out2.out")

Document DataFrame:


,document
0,"Lately I've been dealing with congestion and difficulty breathing, especially while eating. After getting the influenza vaccine, I developed a fever and decreased appetite. Despite trying albuterol treatments, my symptoms haven't improved and my urine output has decreased. Hoping for a solution soon."


Segments DataFrame:


,speaker,begin,end,text
0,SPEAKER_00,0.100,4.940,"Lately I've been dealing with congestion and difficulty breathing, especially while eating."
1,SPEAKER_00,5.901,10.880,"After getting the influenza vaccine, I developed a fever and decreased appetite."
2,SPEAKER_00,11.801,15.898,"Despite trying albuterol treatments, my symptoms haven't improved"
3,SPEAKER_00,16.481,18.810,and my urine output has decreased.
4,SPEAKER_00,19.693,20.999,Hoping for a solution soon.


NER Predictions DataFrame:


,ner_chunk,begin,end,ner_label,ner_confidence
0,congestion,30,39,Symptom,0.9786
1,difficulty breathing,45,64,Symptom,0.79995
2,influenza vaccine,110,126,Vaccine_Name,0.93735003
3,fever,143,147,VS_Finding,0.9875
4,decreased appetite,153,170,Symptom,0.84305
5,albuterol,188,196,Drug_Ingredient,0.9841
6,urine output has decreased,246,271,Symptom,0.41342497
7,solution,287,294,Form,0.9779


Assertion Predictions DataFrame:


,ner_chunk,begin,end,ner_label,assertion
0,congestion,30,39,Symptom,Present
1,difficulty breathing,45,64,Symptom,Present
2,influenza vaccine,110,126,Vaccine_Name,Planned
3,fever,143,147,VS_Finding,Present
4,decreased appetite,153,170,Symptom,Present
5,albuterol,188,196,Drug_Ingredient,Past
6,urine output has decreased,246,271,Symptom,Hypothetical


Relation Predictions DataFrame:


""


#### Example 3

In [37]:
process_s3_output_and_save(validation_file_name3, "out3.out")

Document DataFrame:


,document
0,There is pain in the muscles I do not know caused.


Segments DataFrame:


,speaker,begin,end,text
0,SPEAKER_00,0.523,3.819,There is pain in the muscles I do not know caused.


NER Predictions DataFrame:


,ner_chunk,begin,end,ner_label,ner_confidence
0,pain,9,12,Symptom,0.9794
1,muscles,21,27,Internal_organ_or_component,0.3696


Assertion Predictions DataFrame:


,ner_chunk,begin,end,ner_label,assertion
0,pain,9,12,Symptom,Absent


Relation Predictions DataFrame:


""


In [38]:
model.delete_model()

INFO:sagemaker:Deleting model with name: speech-to-text-medical-2024-04-24-20-01-08-209


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

